In [1]:
import cv2
from cv2 import aruco
import numpy as np
import time
import serial
import time

# Arduinoとのシリアル通信設定
arduino_port = 'COM4'  # Arduinoのポートに合わせて変更
arduino_baudrate = 115200
ser = serial.Serial(arduino_port, arduino_baudrate, timeout=1)

class MarkSearch:
    def __init__(self, cameraID):
        self.cap = cv2.VideoCapture(cameraID)
        self.dict_aruco = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
        self.parameters = aruco.DetectorParameters()

    def get_mark_coordinate(self, num_id):
        ret, frame = self.cap.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        corners, ids, rejectedImgPoints = aruco.detectMarkers(gray, self.dict_aruco, parameters=self.parameters)

        if ids is not None and num_id in ids:
            index = np.where(ids == num_id)[0][0]
            cornerUL = corners[index][0][0]
            cornerUR = corners[index][0][1]
            cornerBR = corners[index][0][2]
            cornerBL = corners[index][0][3]

            center = [(cornerUL[0] + cornerBR[0]) / 2, (cornerUL[1] + cornerBR[1]) / 2]

            # Draw coordinates on the frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            cv2.putText(frame, f'ID: {num_id}', (int(center[0]), int(center[1])), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
            cv2.putText(frame, f'Coord: ({center[0]:.2f}, {center[1]:.2f})', (10, 30), font, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

            cv2.imshow('Camera Window', frame)

            return center

        cv2.imshow('Camera Window', frame)

        return None

if __name__ == "__main__":
    cameraID = 0
    cam0_mark_search = MarkSearch(cameraID)

    markID = 5

    try:
        while True:
            coordinates = cam0_mark_search.get_mark_coordinate(markID)
            
            if coordinates is not None:
                print("Python Response:", coordinates)
                x = f"{coordinates[0]:.2f}"
                y = f"{coordinates[1]:.2f}"

                # データを送信
                ser.write((str(x) + ' ' + str(y) + '\n').encode())

                # Arduinoからの応答を待機
                response = ser.readline().decode().strip()
                print("Arduino Response:", response)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    except KeyboardInterrupt:
        cam0_mark_search.cap.release()

    finally:
        # シリアル通信のクローズ
        ser.close()

cv2.destroyAllWindows()


SerialException: could not open port 'COM4': FileNotFoundError(2, '指定されたファイルが見つかりません。', None, 2)